In [24]:
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


In [25]:
dat = pd.read_csv(os.getcwd() + "/data/Question_Classification_Dataset.csv")
col = ["Questions", "Category1"]
dat = dat[col]
dat["Category_id"] = dat["Category1"].factorize()[0]
class_num = dat["Category_id"].unique()

tfidf = TfidfVectorizer(min_df=4, encoding='latin-1', ngram_range=(1, 2), stop_words='english', sublinear_tf=True)
features = tfidf.fit_transform(dat.Questions).toarray()
fitted_vectorizer = tfidf.fit(dat.Questions)
labels = dat.Category_id

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2, random_state=0)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
print(X_train.shape, X_test.shape)

(4361, 1596) (1091, 1596)


In [64]:
def tr_y(x):
    """
    Convert class label in respective vector.
    5 -> [0,0,0,0,0,1]
    """
    empty = [0, 0, 0, 0, 0, 0]
    empty[x] = 1
    return empty

def tr_y_inverse(arr):
    """
    Convert class vector in the class label.
    [0,0,0,0,0,1] -> 5
    """
    return sum([x * i for i, x in enumerate(arr)])


In [46]:
Y_train_vec = np.array([tr_y(y) for y in  Y_train])
Y_test_vec = np.array([tr_y(y) for y in  Y_test])

In [51]:
class N_classifier():
    def __init__(self, train_data, train_class):
        self.train_data = train_data
        self.train_class = train_class
    
    def model(self):
        """
        Neural network architecture.
        """
        # define the keras model
        model = Sequential()
        model.add(Dense(self.train_data.shape[1], input_dim=self.train_data.shape[1], activation='relu'))
        model.add(Dense(self.train_data.shape[1], activation='relu'))
        model.add(Dense(6, activation='sigmoid'))
        # compile the keras model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        # fit the keras model on the dataset
        model.fit(self.train_data, self.train_class, epochs=10, batch_size=10)
        return model
        

In [52]:
nn = N_classifier(X_train, Y_train_vec)
model = nn.model()

Epoch 1/10
4361/4361 [==============================] - 74s 17ms/step - loss: 0.3313 - acc: 0.8670
Epoch 2/10
4361/4361 [==============================] - 68s 16ms/step - loss: 0.1880 - acc: 0.9264
Epoch 3/10
4361/4361 [==============================] - 67s 15ms/step - loss: 0.1273 - acc: 0.9501
Epoch 4/10
4361/4361 [==============================] - 59s 14ms/step - loss: 0.0943 - acc: 0.9635
Epoch 5/10
4361/4361 [==============================] - 59s 14ms/step - loss: 0.0762 - acc: 0.9706
Epoch 6/10
4361/4361 [==============================] - 53s 12ms/step - loss: 0.0653 - acc: 0.9729
Epoch 7/10
4361/4361 [==============================] - 52s 12ms/step - loss: 0.0595 - acc: 0.9745
Epoch 8/10
4361/4361 [==============================] - 56s 13ms/step - loss: 0.0565 - acc: 0.9749
Epoch 9/10
4361/4361 [==============================] - 60s 14ms/step - loss: 0.0540 - acc: 0.9755
Epoch 10/10
4361/4361 [==============================] - 63s 14ms/step - loss: 0.0530 - acc: 0.9757


In [53]:
model.save(os.getcwd() + "/doc/model_dat/model_10ep.h5")
model.save_weights(os.getcwd() + "/doc/model_dat/model_param_10ep.h5")

In [62]:
_, accuracy = model.evaluate(X_test, Y_test_vec)
print('Accuracy: %.2f' % (accuracy*100))

1091/1091 [==============================] - 0s 231us/step
Accuracy: 87.56
